In [30]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from datetime import date
import requests
import pandas as pd
import time

In [31]:
driver = webdriver.Chrome(executable_path=r'C:\Program Files\chromedriver.exe')
driver.get("https://public.tableau.com/en-gb/gallery/?tab=viz-of-the-day&type=viz-of-the-day&page=1")
wait = WebDriverWait(driver, 20)

#creates a file name that can be used in a Windows file path
def valid_file_name(name):
    return name.replace(" ", "_").replace("?","").replace(":","").replace("'","").replace('"','').replace(".","_").replace(";","").replace("ö","oe").replace("/","_").replace("|","_")

date_col = []
viz_col = []
valid_name_col = []
thumbnail = []

while True:

    vizzes = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".gallery-items-list div.gallery-list-item-container")))
    
    for viz in vizzes:

        #published date 
        date_id = viz.find_element_by_css_selector('[data-test-id="published-date"]').text
        date_col.append(date_id)

        #name of the viz
        viz_name = viz.find_element_by_class_name("gallery-list-item-title-left").text
        viz_col.append(viz_name)
        
        #The file name which is used for the thumbnails
        valid_name = valid_file_name(viz_name)
        valid_name_col.append(valid_name)


        #Get image links
        try:
            images = viz.find_element_by_xpath(".//img[@data-test-id='galleryListItem-thumbnail-image']")
            image_link = images.get_attribute("src")
            
            #download images 
            myfile = requests.get(image_link)

            with open("G:\\My Drive\\Python\\VotD\\Images" + "\\" + valid_file_name(viz_name) + ".jpg", "wb") as f:
                f.write(myfile.content)
            
            thumbnail.append("Y")
        
        #Some of the older vizzes don't have a thumbnail and will be skipped  
        except:
            thumbnail.append("N")
            continue
        
    time.sleep(10)
    
    #xpath is identical for "previous" and "next" button: click second instance of button on second to penultimate page
    try:
        button = driver.find_elements_by_xpath("//li[@class='pagination-end']//a[@tabindex='0']")[1]
        if button.text == "next >":
            button.click()
        else:
            break
    
    #Click the only "next" button on first and last page. Break out of code on last page and close the driver.
    except:
        button = driver.find_elements_by_xpath("//li[@class='pagination-end']//a[@tabindex='0']")[0]
        if button.text == "next >":
            button.click()
        else:
            break
            
time.sleep(5)

driver.close()         

In [32]:
df = pd.DataFrame({"Date": date_col, "Viz": viz_col, "Thumbnail name": valid_name_col, "Thumbnail?": thumbnail})

In [33]:
df["Date"] = df["Date"].str.lstrip("Featured On: ")

In [34]:
df["Date"] = pd.to_datetime(df["Date"], format = "%d %B %Y") 

In [35]:
df

,Date,Viz,Thumbnail name,Thumbnail?
0,2020-02-28,The United States House of Representatives,The_United_States_House_of_Representatives,Y
1,2020-02-27,How has the world changed since 1962?,How_has_the_world_changed_since_1962,Y
2,2020-02-25,Deprivation and Childhood Obesity in England,Deprivation_and_Childhood_Obesity_in_England,Y
3,2020-02-24,How is the world's wealth distributed?,How_is_the_worlds_wealth_distributed,Y
4,2020-02-21,Crisis Text Line Trends in the United States,Crisis_Text_Line_Trends_in_the_United_States,Y
...,...,...,...,...
1619,2012-05-03,Cash Buyers Dominate,Cash_Buyers_Dominate,N
1620,2012-04-29,Case-Shiller Home Price Index,Case-Shiller_Home_Price_Index,Y
1621,2012-04-28,The Worlds Nuclear Reactors,The_Worlds_Nuclear_Reactors,Y
1622,2012-04-27,Best Paid Sports Teams,Best_Paid_Sports_Teams,Y


In [36]:
df.to_csv("G:\\My Drive\\Python\\VotD\\VOTD_list.csv", index = False)